## All in One

In [1]:
from joblib import Parallel, delayed
from tqdm import tqdm
import numpy as np
from pathlib import Path
import pandas as pd

filename = "aisdk-2023-11-08_1_kkn"

data =  pd.read_csv(f"../workdir/AIS-KNN-Files/{filename}.csv", lineterminator='$').values.reshape((-1,100,6))

multiindex = pd.MultiIndex.from_product([range(s) for s in data.shape], names=["track", "timeindex", "column"])
data = pd.Series(data.flatten(), index=multiindex).unstack(level="column")
data.columns = ["mmsi", "time", "x", "y", "deg", "dist"]
data = data.drop(columns=["deg", "dist"])
data["time"] = pd.to_datetime(data["time"], unit="s")
train_ships = pd.Series(data.mmsi.unique()).sample(frac=0.7, random_state=42)
train = data[data.mmsi.isin(train_ships)].drop(columns="mmsi")
test = data[~data.mmsi.isin(train_ships)].drop(columns="mmsi")

data = []
train_ships = []

def generate_row(df):
    df = df.copy()
    df["src"] = "output"
    df["time"] = (df["time"].max()-df["time"]).dt.total_seconds()
    df["x"] = df["x"] - df["x"].iloc[0]
    df["y"] = df["y"] - df["y"].iloc[0]
    df.iloc[:-1,-1] = "input_" + pd.Series(np.arange(df.shape[0]-1)).astype(str)
    df=df.melt(id_vars="src")
    df["colname"] = df.src + "_" + df.variable
    df=df.drop(columns=["src", "variable"]).set_index("colname")
    df=df.transpose().reset_index(drop=True)
    df.columns.name = None
    df=df.sort_index(axis=1).drop(["output_time"], axis=1)
    df=df.loc[:, ~df.columns.to_series().str.endswith('index')]
    return df

def generate_model_frame(df, known):
    window_size = known + 1
    skipped_windows = 4

    windows = df.reset_index(drop=False).groupby('track').rolling(window_size)
    windows = (win for i, win in enumerate(windows) if len(win) == window_size and i % (skipped_windows + 1) == 0)
    windows = list(windows)
    res = Parallel(n_jobs=-1)(delayed(generate_row)(win) for win in tqdm(windows))
    return pd.concat(res, ignore_index=True)

for known in range(12, 2, -1):
    path = Path(f"../workdir/AIS-ModelFrames/{filename}_train_{known}_0.csv")
    if not path.exists():     
        train_model = generate_model_frame(train, known)
        test_model = generate_model_frame(test, known)

        if known == 12:   
            var = list(train_model.shape)
            var = var[0]
            trainlimit = var
            print(f"Trainlimit: {trainlimit}")

            var = list(test_model.shape)
            var = var[0]
            testlimit = var
            print(f"Testlimit: {testlimit}")

        print(train_model.shape)
        train_model = train_model.sample(n = trainlimit)
        print(train_model.shape)

        print(test_model.shape)
        test_model = test_model.sample(n = testlimit)
        print(test_model.shape)
        
        train_model.to_csv(path, index=False)
        test_model.to_csv(f"../workdir/AIS-ModelFrames/{filename}_test_{known}_0.csv", index=False)
        print(f"Done with {known}")

  0%|          | 20/1112344 [00:00<10:29:15, 29.46it/s]/home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
100%|██████████| 1112344/1112344 [12:13<00:00, 1516.99it/s]


Trainlimit: 2681903
Testlimit: 1112344
(2681903, 38)
(2681903, 38)
(1112344, 38)
(1112344, 38)
Done with 12


100%|██████████| 1112344/1112344 [11:55<00:00, 1553.94it/s]


(2681903, 35)
(2681903, 35)
(1112344, 35)
(1112344, 35)
Done with 11


100%|██████████| 1177776/1177776 [12:36<00:00, 1556.22it/s]


...

(2997421, 17)
(2681903, 17)
(1243208, 17)
(1112344, 17)
Done with 5


100%|██████████| 1243208/1243208 [13:16<00:00, 1560.73it/s]


(2997421, 14)
(2681903, 14)
(1243208, 14)
(1112344, 14)
Done with 4


100%|██████████| 1243208/1243208 [13:15<00:00, 1563.41it/s]


(2997421, 11)
(2681903, 11)
(1243208, 11)
(1112344, 11)
Done with 3
